In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Flatten , Dense , Dropout , Activation
from keras.layers.convolutional import Convolution2D , MaxPooling2D , ZeroPadding2D
from keras.losses import categorical_crossentropy
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import EarlyStopping
import cv2
import os
from sklearn.model_selection import train_test_split
os.chdir("C:/Users/Ruchita/Desktop/Project_Dataset/")

In [2]:
age_gender_data = pd.read_csv('age_gender.csv')

In [3]:
age_gender_data

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...
...,...,...,...,...,...
23700,99,0,1,20170120221920654.jpg.chip.jpg,127 100 94 81 77 77 74 99 102 98 128 145 160 1...
23701,99,1,1,20170120134639935.jpg.chip.jpg,23 28 32 35 42 47 68 85 98 103 113 117 130 129...
23702,99,2,1,20170110182418864.jpg.chip.jpg,59 50 37 40 34 19 30 101 156 170 177 184 187 1...
23703,99,2,1,20170117195405372.jpg.chip.jpg,45 108 120 156 206 197 140 180 191 199 204 207...


In [4]:
pixels = []
gender = []

for index , row in age_gender_data.iterrows():
    k = row['pixels'].split(" ")
    pixels.append(np.array(k ,'float32'))
    gender.append(row['gender'])

pixels_train = []
gender_train = []
pixels_test = []
gender_test = []

pixels_train , pixels_test , gender_train , gender_test = train_test_split(pixels , gender , test_size=0.2 , random_state = 21)

In [5]:
pixels_train = np.array(pixels_train , 'float32')
pixels_test = np.array(pixels_test , 'float32')
gender_train = np.array(gender_train , 'float32')
gender_test = np.array(gender_test , 'float32')

pixels_train = pixels_train.reshape(pixels_train.shape[0] , 48 , 48 , 1)
pixels_test = pixels_test.reshape(pixels_test.shape[0] , 48 , 48 , 1)

gender_train = to_categorical(gender_train , num_classes = 2)
gender_test = to_categorical(gender_test , num_classes = 2)

In [6]:
model = Sequential()
es=EarlyStopping(patience=10)

model.add(Convolution2D(input_shape = (48 , 48 , 1) , filters = 64 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 64 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 128 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 128 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Flatten())
model.add(Dense(units = 4096 , activation = 'relu'))
model.add(Dense(units = 4096 , activation = 'relu'))
model.add(Dense(units = 2 , activation = 'sigmoid'))

In [7]:
model.compile(optimizer = 'SGD' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [8]:
model.fit(pixels_train , gender_train , batch_size = 64 , epochs = 40 , validation_data = (pixels_test , gender_test),validation_split=0.2,callbacks=es)

Epoch 1/40
238/238 [==============================] - 1150s 5s/step - loss: 0.6621 - accuracy: 0.6088 - val_loss: 0.6094 - val_accuracy: 0.6678
Epoch 2/40
238/238 [==============================] - 1271s 5s/step - loss: 0.5927 - accuracy: 0.6867 - val_loss: 0.6033 - val_accuracy: 0.6971
Epoch 3/40
238/238 [==============================] - 1231s 5s/step - loss: 0.4696 - accuracy: 0.7809 - val_loss: 0.9824 - val_accuracy: 0.4756
Epoch 4/40
238/238 [==============================] - 1154s 5s/step - loss: 0.3934 - accuracy: 0.8219 - val_loss: 0.4440 - val_accuracy: 0.8173
Epoch 5/40
238/238 [==============================] - 1207s 5s/step - loss: 0.3415 - accuracy: 0.8465 - val_loss: 0.7664 - val_accuracy: 0.6562
Epoch 6/40
238/238 [==============================] - 1200s 5s/step - loss: 0.3132 - accuracy: 0.8583 - val_loss: 0.9210 - val_accuracy: 0.5191
Epoch 7/40
238/238 [==============================] - 1203s 5s/step - loss: 0.2950 - accuracy: 0.8713 - val_loss: 0.3348 - val_accuracy:

In [9]:
loss_and_metrics = model.evaluate(pixels_test , gender_test)
print(loss_and_metrics)

149/149 [==============================] - 84s 562ms/step - loss: 0.3607 - accuracy: 0.8933
[0.36072733998298645, 0.8932714462280273]


In [10]:
model_json = model.to_json()
with open("model_gender.json" , "w") as json_file:
    json_file.write(model_json)
    model.save_weights("model_gender.h5")
    print("Saved model to disk")

Saved model to disk
